In [4]:
import cv2
import numpy as np
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, LeakyReLU, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

class Meso4:
    def __init__(self):
        self.model = self.create_model()

    def create_model(self):
        # Input shape matches the resized images
        inputs = Input(shape=(112, 112, 3))
        
        # First convolutional block
        x = Conv2D(8, (3, 3), padding='same', activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # Output: (56, 56, 8)

        # Second convolutional block
        x = Conv2D(16, (5, 5), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # Output: (28, 28, 16)

        # Third convolutional block
        x = Conv2D(32, (5, 5), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # Output: (14, 14, 32)

        # Fourth convolutional block
        x = Conv2D(64, (5, 5), padding='same', activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)  # Output: (7, 7, 64)

        # Flatten and Dense layers
        x = Flatten()(x)
        x = Dropout(0.5)(x)
        x = Dense(64)(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = Dropout(0.5)(x)
        x = Dense(1, activation='sigmoid')(x)

        return Model(inputs=inputs, outputs=x)
    
    def compile(self):
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def train(self, train_generator, validation_generator, epochs=10):
        return self.model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=epochs
        )

    def save_weights(self, path):
        self.model.save_weights(path)
        print(f"Weights saved to {path}")

    def load_weights(self, path):
        self.model.load_weights(path)
        print(f"Weights loaded from {path}")

    def predict_frame(self, frame):
        # Preprocess the frame (resize, normalize, etc.)
        frame_resized = tf.image.resize(frame, (112, 112))  # Resize to match model input
        frame_resized = frame_resized / 255.0  # Normalize
        frame_resized = np.expand_dims(frame_resized, axis=0)  # Add batch dimension
        
        prediction = self.model.predict(frame_resized)
        return prediction[0][0]

    def process_video(self, video_path):
        # Open video
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print("Error: Could not open video file.")
            return []

        deepfake_count = 0
        total_frames = 0
        predictions = []

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to read a frame or end of video.")
                break  # Break out of the loop if no more frames are read

            total_frames += 1
            print(f"Processing frame {total_frames}...")

            # Preprocess frame
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
            prediction = self.predict_frame(frame_rgb)

            predictions.append(prediction)

            if prediction > 0.5:
                deepfake_count += 1

        cap.release()

        if total_frames == 0:
            print("No frames were processed.")
            return []

        # Output
        print(f"Total Frames: {total_frames}")
        print(f"Deepfake Frames: {deepfake_count}")
        print(f"Non-Deepfake Frames: {total_frames - deepfake_count}")
        
        deepfake_percentage = (deepfake_count / total_frames) * 100
        print(f"Percentage of Deepfake Frames: {deepfake_percentage:.2f}%")
        return predictions

# Initialize and compile the model
meso = Meso4()
meso.compile()

# Load data using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Use 20% of data for validation
)

# Path to the data folder containing 'real' and 'deepfake' subfolders
data_dir = 'data'

# Training data generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(112, 112),  # Resize images to match model input
    batch_size=32,
    class_mode='binary',  # Binary classification (real vs deepfake)
    subset='training'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(112, 112),  # Resize images to match model input
    batch_size=32,
    class_mode='binary',  # Binary classification (real vs deepfake)
    subset='validation'
)

# Train the model
history = meso.train(
    train_generator,
    validation_generator,
    epochs=4  # Train for 20 epochs
)

# Save the model weights
weights_path = 'hackanova/Deepfake-detection/weights/Meso4_DF.weights.h5'
os.makedirs(os.path.dirname(weights_path), exist_ok=True)
meso.save_weights(weights_path)

# Process a video and predict whether it contains deepfake frames


Found 5684 images belonging to 2 classes.
Found 1420 images belonging to 2 classes.


c:\Users\Harshav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\Harshav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/4
178/178 ━━━━━━━━━━━━━━━━━━━━ 129s 707ms/step - accuracy: 0.8419 - loss: 0.5154 - val_accuracy: 0.7585 - val_loss: 0.4257
Epoch 2/4
178/178 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.9588 - loss: 0.1118 - val_accuracy: 0.8092 - val_loss: 0.3869
Epoch 3/4
178/178 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.9711 - loss: 0.0831 - val_accuracy: 0.8451 - val_loss: 0.5599
Epoch 4/4
178/178 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - accuracy: 0.9847 - loss: 0.0442 - val_accuracy: 0.8387 - val_loss: 0.6337
Weights saved to hackanova/Deepfake-detection/weights/Meso4_DF.weights.h5


In [9]:
video_path = "download (3).mp4"
predictions = meso.process_video(video_path)

Processing frame 1...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Processing frame 2...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 3...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing frame 4...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processing frame 5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 6...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing frame 7...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processing frame 8...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 9...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processing frame 10...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing frame 11...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Processing frame 12...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 13...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 14...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processing frame 15...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing frame 16...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Processing frame 17...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms

In [8]:
    # Save the model (including architecture, optimizer, and weights)
model_path = 'models/Meso4_DF_model.h5'
os.makedirs(os.path.dirname(model_path), exist_ok=True)
meso.model.save(model_path)



